In [1]:
import numpy as np
import pandas as pd 
from pathlib import Path, PurePath
import nltk
from nltk.corpus import stopwords
import string
import torch
from transformers import BertTokenizer, AutoTokenizer, BertForQuestionAnswering, AutoModelForQuestionAnswering
import glob
import os
import json
from collections import Counter
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords as nltkstopwords
from nltk.tokenize import word_tokenize
import re
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_numeric, remove_stopwords, strip_multiple_whitespaces
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /home/nic/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/nic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
unable to import 'smart_open.gcs', disabling that module


In [ ]:
base_path = "../input/CORD-19-research-challenge"

biorxiv_medrxiv = "biorxiv_medrxiv/biorxiv_medrxiv/pdf_json"
comm_use_subset = "comm_use_subset/comm_use_subset/pdf_json"
noncomm_use_subset = "noncomm_use_subset/noncomm_use_subset/pdf_json"
custom_license = "custom_license/custom_license/pdf_json"

In [2]:
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import pipeline

# load BART summarizer
summarizer = pipeline(task="summarization")


Couldn't reach server at 'https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large-cnn/modelcard.json' to download model card file.
Creating an empty model card.


(47298, 5)
(35901, 5)


,title,doi,abstract,publish_time,authors
0,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,abstract the etiologic basis for the vast majo...,1972-12-31,"Overall, James C."
3,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,"abstract middle-aged female identical twins, o...",1973-08-31,"Brunner, Carolyn M.; Horwitz, David A.; Shann,..."
4,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,abstract upper respiratory tract infections ar...,1985-06-28,"Garibaldi, Richard A."
5,Infectious diarrhea: Pathogenesis and risk fac...,10.1016/0002-9343(85)90367-5,abstract our understanding of the pathogenesis...,1985-06-28,"Cantey, J.Robert"
6,New perspectives on the pathogenesis of rheuma...,10.1016/0002-9343(88)90356-7,abstract in the pathogenesis of rheumatoid art...,1988-10-14,"Zvaifler, Nathan J."


In [4]:
# load the meta data from the CSV file using 3 columns (abstract, title, authors),
df = pd.read_csv(
    "../input/CORD-19-research-challenge/metadata.csv",
    usecols=["title", "abstract", "authors", "doi", "publish_time"],
)
print(df.shape)
# drop duplicates
# df=df.drop_duplicates()
df = df.drop_duplicates(subset="abstract", keep="first")
# drop NANs
df = df.dropna()
# convert abstracts to lowercase
df["abstract"] = df["abstract"].str.lower()
# show 5 lines of the new dataframe
print(df.shape)
df.head()

(47298, 5)
(35901, 5)


,title,doi,abstract,publish_time,authors
0,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,abstract the etiologic basis for the vast majo...,1972-12-31,"Overall, James C."
3,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,"abstract middle-aged female identical twins, o...",1973-08-31,"Brunner, Carolyn M.; Horwitz, David A.; Shann,..."
4,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,abstract upper respiratory tract infections ar...,1985-06-28,"Garibaldi, Richard A."
5,Infectious diarrhea: Pathogenesis and risk fac...,10.1016/0002-9343(85)90367-5,abstract our understanding of the pathogenesis...,1985-06-28,"Cantey, J.Robert"
6,New perspectives on the pathogenesis of rheuma...,10.1016/0002-9343(88)90356-7,abstract in the pathogenesis of rheumatoid art...,1988-10-14,"Zvaifler, Nathan J."


In [10]:
import functools
from IPython.core.display import display, HTML
from nltk import PorterStemmer
from tqdm.notebook import tqdm_notebook

tqdm_notebook.pandas()

# list of lists for topic words realting to tasks
display(HTML("<h1>COVID-19 Risk Factors</h1>"))
display(
    HTML(
        "<h3>Table of Contents (ctrl f to search for the hash tagged words below to find that data table)</h3>"
    )
)
tasks = [
    ["comorbidities"],
    ["risk", "factors"],
    ["lung", "cancer"],
    ["hypertension"],
    ["heart", "disease"],
    ["chronic", "bronchitis"],
    ["cerebral", "infarction"],
    ["diabetes"],
    ["copd"],
    ["blood type", "type"],
    ["smoking"],
    ["effective", "reproductive", "number"],
    ["incubation", "period", "days"],
]


def summarize(row):
    summary = ""

    abstract = row

    summary = summarizer(abstract, min_length=50, max_length=200)
    summary = summary[0]["summary_text"]

    if summary != "":
        print(summary)


# function to stem keywords into a common base word
def stem_words(words):
    stemmer = PorterStemmer()
    singles = []
    for w in words:
        singles.append(stemmer.stem(w))
    return singles

print(df1.loc[0])
summarize(body)

filename = str1 + ".csv"
df_table.to_csv(filename, index=False)
df_table = HTML(df_table.to_html(escape=False, index=False))
display(df_table)

index                                                        5973
title           Kidney disease is associated with in-hospital ...
doi                                    10.1016/j.kint.2020.03.005
abstract        abstract in december 2019, a coronavirus 2019 ...
publish_time                                           2020-03-20
authors         Cheng, Yichun; Luo, Ran; Wang, Kun; Zhang, Men...
Name: 0, dtype: object
Strongyloides stercoralis infection particularly related to immunosuppressed patients although systematic screening has not been widely implemented at national level it was established that individuals either hospitalized or be uals seems to be lower but these are limited data. The high prevalence found in our study supports the need of screening strategies in patients that are potentially immunos Puppressed.


AttributeError: 'HTML' object has no attribute 'to_csv'

In [8]:
body="""referent hospitals in spain hospital clinic hospital del mar hospital universitari bellvitge hospital sant pau and hospital vall dhebron located in barcelona and hospital de poniente in almeria province aimed to evaluate the prevalence and risk factors associated with strongyloides stercoralis infection particularly related to immunosuppressed patients although systematic screening has not been widely implemented at national level it was established that individuals either hospitalized or be uals seems to be lower but these are limited data and further prospective studies should better evaluate the accuracy of the serological tests in immunosuppressed patients even with the limitations of current diagnostic methods screening of highrisk groups and treatment of infected individuals is of key importance in this regard screening of strongyloidiasis in newly arrived migrants has been recommended by the european centre for disease prevention and control particularly in immu preprint the high prevalence found in our study supports the need of screening strategies in patients that are potentially immunosuppressed particularly for subsaharan african and latin american migrants potentially immunosuppressed people are at higher risk of developing severe complication of the disease as it has been extensively reported elsewhere however the probability of developing hyperinfection or dissemination of the infection in the highrisk immunosuppressed population is unclear ntially immunosuppressed people are at higher risk of developing severe complication of the disease as it has been extensively reported elsewhere however the probability of developing hyperinfection or dissemination of the infection in the highrisk immunosuppressed population is unclear systematic screening has been recently recommended in highrisk population coming from endemic areas however few programmes have reported their results so far in this regard reference centre in aus it has been extensively reported elsewhere however the probability of developing hyperinfection or dissemination of the infection in the highrisk immunosuppressed population is unclear systematic screening has been recently recommended in highrisk population coming from endemic areas however few programmes have reported their results so far in this regard reference centre in austria has initiated screening of sstercoralis to all transplant recipients showing prevalence even th ry distress syndrome ards multiple organ dysfunction syndrome mods and death so the top priority for clinicians is to identify and treat the severest patients in the early stage the aim of this study was to assess the potential highrisk factors of 2019ncov severe pneumonia and provide evidence for the screening of severely afflicted patients ccbyncnd international license it is made available under is the authorfunder who has granted medrxiv license to display the prepri or they were described using the median the mannwhitney test was used for comparisons categorical variables were expressed as count and compared by χ² test or fishers exact test logistic regression analysis was used to assess the risk factors of severe pneumonia the difference of certain indicator in the same patient at different periods was shown by bar chart twosided of less than was considered statistically significant we used spss software version fo the copyright holder for this preprint httpsdoiorg1011012020030220029306 doi medrxiv preprint according to the results of the binomial logistics regression analysis we found that age and the level of the ddimer were independent risk factors these results suggested that the level of ddimer was significantly positively correlated with the 2019ncov severe pneumonia which was also shown in another study previous studies showed that sarscov could bind to ace2 downregu iratory distress syndrome ards multiple organ dysfunction syndrome mods and death so the top priority for clinicians is to identify and treat the severest patients in the early stage the aim of this study was to assess the potential highrisk factors of 2019ncov severe pneumonia and provide evidence for the screening of severely afflicted patients ccbyncnd international license it is made available under is the authorfunder who has granted medrxiv license to display the mine whether they exacerbated to severe ncp progressive group or not stable group ct and clinical characteristics between the two groups were compared and multivariate logistic regression and sensitivity analyses were performed to identify the risk factors for developing severe ncp results total of patients with moderate ncp were included of which patients developed severe ncp during hospitalization and assigned to the progressive group multivariate logistic regression analysis"""

In [2]:
input_dir = PurePath('../input/CORD-19-research-challenge')
metadata_path = input_dir / 'metadata.csv'
metadata_df = pd.read_csv(metadata_path, low_memory=False)
metadata_df = metadata_df.dropna(subset=['abstract', 'title']) \
                            .reset_index(drop=True)

In [3]:
base_path = "../input/CORD-19-research-challenge"
biorxiv_medrxiv = "biorxiv_medrxiv/biorxiv_medrxiv/pdf_json"
comm_use_subset = "comm_use_subset/comm_use_subset/pdf_json"
noncomm_use_subset = "noncomm_use_subset/noncomm_use_subset/pdf_json"
custom_license = "custom_license/custom_license/pdf_json"

files = []

def read_files(directory):
    files_in_dir = [f for f in glob.glob(os.path.join(base_path, directory) + "/*.json", recursive=True)]
    files.extend(files_in_dir)
    raw_body_texts_of_file = []
    for f in files_in_dir:
        data = json.load(open(f))
        #for key in data.keys():
        #    print("{}: {}\n".format(key, data[key]))
        
        body_text = ""
        if True:
            for i in range(len(data["abstract"])):
                body_text += " " + data["abstract"][i]["text"]
        else:
            for i in range(len(data["body_text"])):
                body_text += " " + data["body_text"][i]["text"]
        
        body_text = re.sub(' +', ' ', body_text)
        raw_body_texts_of_file.append(body_text)
    return raw_body_texts_of_file

raw_body_texts = []
raw_body_texts.extend(read_files(biorxiv_medrxiv))
raw_body_texts.extend(read_files(comm_use_subset))
#raw_body_texts.extend(read_files(noncomm_use_subset))
#raw_body_texts.extend(read_files(custom_license))
print("Found {} raw body texts".format(len(raw_body_texts)))

Found 10707 raw body texts


In [4]:
NUMBER_OF_FILES = len(raw_body_texts)
WINDOW_SIZE = 200

factor___risk_pattern = r"(factor(.){0,9}risk)" # for example for "factors of risk"
risk_factor_pattern = r"(risk(.){0,4}factor)" # for example risk factors
risk_pattern = r"(risk)"
high_risk_pattern = r"(high(.){0,6}risk)"
comorbdit_pattern = r"(comorbdit)"
co_infects_pattern = r"(co(.){0,4}infect)"
neonat_pattern = r"(neonat)"
pregnant_pattern = r"(pregnant)"
smoking_pattern = r"(smoking)"
cancer_pattern = r"(cancer)"
averse_outcomes_pattern = r"(advers(.){0,4}outcome)"

PATTERNS = [
    factor___risk_pattern,
    risk_factor_pattern,
    #risk_pattern,
    high_risk_pattern,
    #comorbdit_pattern,
    #co_infects_pattern,
    #neonat_pattern,
    #pregnant_pattern,
    #smoking_pattern,
    #cancer_pattern,
    averse_outcomes_pattern
]

CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation]
CUSTOM_FILTERS_EXCLUDE_NUMERIC = [lambda x: x.lower(), strip_tags, strip_punctuation, strip_numeric]


def extract_windows_containing(pattern, print_out=True):
    indices = []
    preprocessed_texts = []
    for idx in range(NUMBER_OF_FILES):
        filtered_sentence = raw_body_texts[idx]
        preprocessed_texts.append(filtered_sentence)
        
        indices_of_file = [(m.start(0), m.end(0)) for m in re.finditer(pattern, filtered_sentence)]
        indices.append(indices_of_file)
    
    return indices, preprocessed_texts


indices = [[] for _ in range(NUMBER_OF_FILES)]
for pattern in PATTERNS:
    indices_, preprocessed_texts = extract_windows_containing(pattern)
    for i in range(len(indices_)):
        indices[i].extend(indices_[i])

print("Found {} candidates".format(len([1 for a in indices if len(a)!=0])))

def process_file(file_number, indices_of_file, filters):
    tokenized_matches = []
    for match in indices_of_file:
        start = match[0]-WINDOW_SIZE
        end = match[1]+WINDOW_SIZE
        text = preprocessed_texts[file_number][start:end]
        #tokenized = preprocess_string(text, filters)
        tokenized_matches.append(text)

    return tokenized_matches


Found 442 candidates


In [33]:

documents = []
for file_number, indices_of_file in enumerate(indices):
    if len(indices_of_file) != 0:
        for data in process_file(file_number, indices_of_file, CUSTOM_FILTERS_EXCLUDE_NUMERIC):
            def remove_special_character( text):
                #Remove special characters from text string
                return text.translate(str.maketrans('', '', string.punctuation))

            def tokenize(text):
                # tokenize text
                words = nltk.word_tokenize(text)
                return [str(word).lower() for word in words if len(word) > 1 and not word.isnumeric()]
            documents.append(" ".join(tokenize(remove_special_character(data))))

oronavirus (2019-nCoV). The clinical characteristics of severe pneumonia caused by 2019-nCoV are 3 still not clear. 4 Objectives: The aim of this study was to explore the clinical characteristics and risk factors of the 5 severe pneumonia caused by the 2019-nCoV in Wuhan, China. 6 Method: The study included patients hospitalized at the central hospital of Wuhan who had been 7 diagnosed with a pneumonia cause
a, laboratory examinations, and chest computed tomography (CT) scans 9 were reviewed through electronic medical records. SPSS was used for data analysis to explore the 10 clinical characteristics and risk factors of the patients with the severe pneumonia.
ance by CT should be recommended to improve clinical outcomes.
an exemplar of a condition where, due to expected or measured abnormalities (hypoxaemia-low PaO2 SpO2), poor physiological capacity might be predicted.  Neither age nor baseline SpO2 predicted lower/higher risk categories by anaerobic threshold, but haemoglobin-depend

In [6]:
temp=0
for a in range(len(documents)):
    if len(" ".join(documents[a])) > temp:
        temp=len(" ".join(documents[a]))
print("max length of document:", temp)

max length of document: 815


In [30]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

SCIBERT = "allenai/scibert_scivocab_uncased"
BERT_SQUAD = 'bert-large-uncased-whole-word-masking-finetuned-squad'
#OTHER = "mrm8488/scibert_scivocab-finetuned-CORD19"
OTHER = "ktrapeznikov/scibert_scivocab_uncased_squad_v2"
#OTHER = "ahotrod/roberta_large_squad2"
#model = BertForQuestionAnswering.from_pretrained(OTHER)
#tokenizer = BertTokenizer.from_pretrained(OTHER)

tokenizer = AutoTokenizer.from_pretrained(OTHER)

model = AutoModelForQuestionAnswering.from_pretrained(OTHER)


model = model.to(torch_device)
model.eval()

print()


def answer_question(question, context):
    encoded_dict = tokenizer.encode_plus(
                        question, context,
                        add_special_tokens = True,
                        max_length = 256,
                        pad_to_max_length = True,
                        return_tensors = 'pt'
                   )
    
    input_ids = encoded_dict['input_ids'].to(torch_device)
    token_type_ids = encoded_dict['token_type_ids'].to(torch_device)
    
    start_scores, end_scores = model(input_ids, token_type_ids=token_type_ids)

    all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)
    
    answer = tokenizer.convert_tokens_to_string(all_tokens[start_index:end_index+1])
    answer = answer.replace('[CLS]', '')
    return answer

In [31]:
NUM_CONTEXT_FOR_EACH_QUESTION = 10

def get_all_context(query, num_results):
    # Return ^num_results' papers that better match the query
    papers_df = cse.search(query, num_results)
    return papers_df['abstract'].str.replace("Abstract", "").tolist()


def get_all_answers(question, all_contexts):
    # Ask the same question to all contexts (all papers)
    
    all_answers = []
    
    for context in all_contexts:
        all_answers.append(answer_question(question, context))
    return all_answers


def create_output_results(question, 
                          all_contexts, 
                          all_answers, 
                          summary_answer='', 
                          summary_context=''):
    # Return results in json format
    
    def find_start_end_index_substring(context, answer):   
        search_re = re.search(re.escape(answer.lower()), context.lower())
        if search_re:
            return search_re.start(), search_re.end()
        else:
            return 0, len(context)
        
    output = {}
    output['question'] = question
    output['summary_answer'] = summary_answer
    output['summary_context'] = summary_context
    results = []
    for c, a in zip(all_contexts, all_answers):

        span = {}
        span['context'] = c
        span['answer'] = a
        span['start_index'], span['end_index'] = find_start_end_index_substring(c,a)

        results.append(span)
    
    output['results'] = results
        
    return output

    
def get_results(question, 
                summarize=False, 
                num_results=NUM_CONTEXT_FOR_EACH_QUESTION,
                verbose=True):
    # Get results

    #all_contexts = get_all_context(question, num_results)
    all_contexts = documents[:NUM_CONTEXT_FOR_EACH_QUESTION]
    #print(len(all_contexts))
    #return
    
    all_answers = get_all_answers(question, all_contexts)
    
    if summarize:
        # NotImplementedYet
        summary_answer = get_summary(all_answers)
        summary_context = get_summary(all_contexts)
    
    return create_output_results(question, 
                                 all_contexts, 
                                 all_answers)

In [32]:
%%time
NUM_CONTEXT_FOR_EACH_QUESTION = 1000
all_tasks = []

for i, t in enumerate(covid_kaggle_questions['data']):
    print("Answering questions to task {}. ...".format(i+1))
    answers_to_question = []
    for q in t['questions']:
            answers_to_question.append(get_results(q, verbose=False))
    task = {}
    task['task'] = t['task']
    task['questions'] = answers_to_question
    
    all_tasks.append(task)

all_answers = {}
all_answers['data'] = all_tasks

Answering questions to task 1. ...
CPU times: user 39.2 s, sys: 49.6 ms, total: 39.3 s
Wall time: 38.6 s


In [26]:
len(documents)

737

In [27]:
from IPython.display import display, Markdown, Latex, HTML

def layout_style():
    style = """
        div {
            color: black;
        }
        .single_answer {
            border-left: 3px solid #dc7b15;
            padding-left: 10px;
            font-family: Arial;
            font-size: 16px;
            color: #777777;
            margin-left: 5px;

        }
        .answer{
            color: #dc7b15;
        }
        .question_title {
            color: grey;
            display: block;
            text-transform: none;
        }      
        div.output_scroll { 
            height: auto; 
        }
    """
    return "<style>" + style + "</style>"

def dm(x): display(Markdown(x))
def dh(x): display(HTML(layout_style() + x))

In [28]:
def display_single_context(context, start_index, end_index):
    
    before_answer = context[:start_index]
    answer = context[start_index:end_index]
    after_answer = context[end_index:]

    content = before_answer + "<span class='answer'>" + answer + "</span>" + after_answer

    return dh("""<div class="single_answer">{}</div>""".format(content))

def display_question_title(question):
    return dh("<h2 class='question_title'>{}</h2>".format(question.capitalize()))


def display_all_contexts(index, question):
    
    def answer_not_found(context, start_index, end_index):
        return (start_index == 0 and len(context) == end_index) or (start_index == 0 and end_index == 0)

    display_question_title(str(index + 1) + ". " + question['question'].capitalize())
    
    # display context
    for i in question['results']:
        if answer_not_found(i['context'], i['start_index'], i['end_index']):
            continue # skip not found questions
        display_single_context(i['context'], i['start_index'], i['end_index'])

def display_task_title(index, task):
    task_title = "Task " + str(index) + ": " + task
    return dh("<h1 class='task_title'>{}</h1>".format(task_title))

def display_single_task(index, task):
    
    display_task_title(index, task['task'])
    
    for i, question in enumerate(task['questions']):
        display_all_contexts(i, question)

In [29]:
task = 1
display_single_task(task, all_tasks[task-1])

In [ ]:
import json
with open("covid_kaggle_answer_from_qa.json", "w") as f:
    json.dump(all_answers, f)